In [1]:
import os
import io
import platform
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn.datasets as datasets
import random
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from PIL import Image
from PIL import ImageOps

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.cl1=nn.Conv2d(3,12,kernel_size=5,padding=2) # tunable here
        self.mp1=nn.MaxPool2d(2)
        self.cl2=nn.Conv2d(12,3,kernel_size=5,padding=2)
        self.mp2=nn.MaxPool2d(2)
        self.fcl=nn.Linear(7500,200) # 12 x 27 x 27
        
    def forward(self,x):
        # x dim: b x 800 x 600 x 3 -> b x 1 x 100 x 100
        out=F.relu(self.cl1(x))
        out=self.mp1(out)
        out=F.relu(self.cl2(out))
        out=self.mp2(out)
        
        # tensor size of b x inchannels x 1 x 1
        out=out.view(out.size(0),-1)
        out=F.sigmoid(self.fcl(out))
        return out

In [3]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean(torch.pow(euclidean_distance, 2) +
                                      torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

#         loss_contrastive=torch.mean(torch.pow(euclidean_distance,2))
        return loss_contrastive

In [4]:
def load_model(path,model):
    model.load_state_dict(torch.load(path))
    return model

In [8]:
cnn=CNN()
load_model('./model.pt',cnn)

CNN(
  (cl1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (mp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cl2): Conv2d(12, 3, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (mp2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fcl): Linear(in_features=7500, out_features=200, bias=True)
)

In [9]:
img1=transforms.ToTensor()(Image.open('./LAG/stas_mikhailov/Aprile_01891_1.png').convert('RGB'))
img2=transforms.ToTensor()(Image.open('./LAG/stas_mikhailov/y/Aprile_01891_2.png').convert('RGB'))
p1=model(img1.unsqueeze(0))
p2=model(img2.unsqueeze(0))
loss=criterion(p1,p2)

CNN(
  (cl1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (mp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cl2): Conv2d(12, 3, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (mp2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fcl): Linear(in_features=7500, out_features=200, bias=True)
)